In [1]:
import copy as pyCopy
%pylab inline
import cv2 as cv2
print(cv2.__version__)

Populating the interactive namespace from numpy and matplotlib
4.1.1


In [9]:
def cubeDetector(img, h_thresh, s_thresh, v_thresh, drawnImg = None, c_color = (255,255,255)):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    
    #cv2.imwrite("flowchart_stuff/hsv.jpg", hsv)
    
    channels = []
    channels = cv2.split(hsv, channels)
    # 0 = H, 1 = S, 2 = V
    
    cv2.imshow("H", cv2.resize(channels[0], (480, 640)))
    cv2.imshow("S", cv2.resize(channels[1], (480, 640)))
    cv2.imshow("V", cv2.resize(channels[2], (480, 640)))
    
    #cv2.imwrite("flowchart_stuff/goldPreS.jpg", channels[1])
    
    channels[0] = cv2.inRange(channels[0], h_thresh[0], h_thresh[1], channels[0]);
    channels[1] = cv2.inRange(channels[1], s_thresh[0], s_thresh[1], channels[1]);
    channels[2] = cv2.inRange(channels[2], v_thresh[0], v_thresh[1], channels[2]);
    
    #cv2.imwrite("flowchart_stuff/goldS.jpg", channels[1])
    
    #cv2.imshow("goldH", cv2.resize(channels[0], (320, 480)))
    #cv2.imshow("goldS", cv2.resize(channels[1], (320, 480)))
    #cv2.imshow("goldV", cv2.resize(channels[2], (320, 480)))

    finalMap = cv2.bitwise_and(channels[0], channels[1])
    finalMap = cv2.bitwise_and(finalMap, channels[2])
    
    cv2.imshow("combined", cv2.resize(finalMap, (480, 640)))
    
    #cv2.imwrite("flowchart_stuff/goldBinary.jpg", finalMap)

    #floodFillMap = pyCopy.deepcopy(finalMap)

    #cv2.floodFill(floodFillMap,None,(0,0),255)

    #floodFillMap = cv2.bitwise_not(floodFillMap)

    #finalMap = cv2.bitwise_or(finalMap, floodFillMap)

    #finalMap = cv2.GaussianBlur( finalMap, (9, 9), 2, 2)
    
    #cv2.imwrite("flowchart_stuff/goldBinaryBlur.jpg", finalMap)
    
    #cv2.imshow("cubeBinaryMap", cv2.resize(finalMap, (480, 640)))
    
    finalMapContours = None;
    finalMapContours, h = cv2.findContours(finalMap, cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE, finalMapContours)
    
    if drawnImg is not None:
        retImg = pyCopy.deepcopy(drawnImg)
    else:
        retImg = pyCopy.deepcopy(img)
    
    if finalMapContours is not None:
        polygons = [];
        for contour in finalMapContours[1]:
            if cv2.contourArea(contour) > 100000:
                polygons.append(cv2.approxPolyDP(contour, .05*cv2.arcLength(contour, True), True))
                M = cv2.moments(contour)
                cX = int(M["m10"] / M["m00"])
                cY = int(M["m01"] / M["m00"])
                 
                # draw the contour and center of the shape on the image
                cv2.circle(retImg, (cX, cY), 50,  (255,255,255) , -1)
                
        cv2.drawContours(retImg, polygons, -1, c_color, 20)
    
    return retImg

In [10]:
frame = cv2.imread("blocks1.jpg")
fs = cv2.resize(frame, (540,540))
orange_thresh = ((0, 30), (100, 256), (190, 256))
frame = cubeDetector(frame, orange_thresh[0], orange_thresh[1], orange_thresh[2], c_color = (0,0, 255))
purple_thresh = ((130, 170), (30, 265), (65, 256))
#frame = cubeDetector(frame, purple_thresh[0], purple_thresh[1], purple_thresh[2], c_color = (255,0, 0))
green_thresh = ((50, 90), (70, 256), (60, 256))
#frame = cubeDetector(frame, green_thresh[0], green_thresh[1], green_thresh[2], c_color = (0, 255, 0))

cv2.imshow("final", cv2.resize(frame, (480, 640)))
while(cv2.waitKey(0) != ord('q')):
    pass
cv2.destroyAllWindows()

error: /home/travis/miniconda/conda-bld/conda_1486587069159/work/opencv-3.1.0/modules/python/src2/cv2.cpp:163: error: (-215) The data should normally be NULL! in function allocate


In [108]:
cap = cv2.VideoCapture(0)
#for i in range (1,2):
while True:
    ret, frame = cap.read()
    
    orange_thresh = ((0, 30), (100, 256), (190, 256))
    ret_img = cubeDetector(frame, orange_thresh[0], orange_thresh[1], orange_thresh[2], c_color = (0,0, 255))
    purple_thresh = ((130, 170), (30, 265), (65, 256))
    ret_img = cubeDetector(frame, purplqe_thresh[0], purple_thresh[1], purple_thresh[2], c_color = (255,0, 0))
    green_thresh = ((50, 90), (70, 256), (60, 256))
    ret_img = cubeDetector(ret_img, green_thresh[0], green_thresh[1], green_thresh[2], c_color = (0, 255, 0))
    
    #whiteMinerals = detectWhiteMineral(pyCopy.deepcopy(frame))
    #cv2.imshow("Img", cubeDetector(frame, drawnImg = pyCopy.deepcopy(whiteMinerals)))
    
    cv2.imshow("img", cv2.resize(ret_img, (480, 640)))

    key = cv2.waitKey(0) & 0xFF
 
    #if the 'q' key is pressed, stop the loop
    if key == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()